In [2]:
import pandas as pd
import numpy as np


In [3]:
ratingDF = pd.read_csv(R"rating.csv")
ratingDF.columns

Index(['id', 'flyer_user_id', 'ad_id', 'rating', 'comments', 'status',
       'fc_given', 'updated_date', 'coupon_redeem_id', 'store_id', 'order_id',
       'store_rating', 'deal_question_ids'],
      dtype='object')

In [4]:
ratings_dict = {'ItemID': list(ratingDF.ad_id),
                'UserID': list(ratingDF.flyer_user_id),
                'Rating': list(ratingDF.rating)}

In [5]:
ratingDf = pd.DataFrame(ratings_dict)

In [6]:
sandf=ratingDf.groupby(['UserID', 'ItemID']).max()['Rating'].reset_index()

In [7]:
rating_df = sandf.pivot(index = 'UserID', columns ='ItemID', values = 'Rating').fillna(0)

In [9]:
R = rating_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\Users\Gandiv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [10]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [11]:
sigma = np.diag(sigma)

In [12]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = rating_df.columns)

In [13]:
userIDList=rating_df.index.values.tolist()
preds_df['UserID']=userIDList
predDF=preds_df.set_index('UserID')

In [14]:
advertisement=pd.read_csv(R'E:\freezon\fyndus\data\advertisement.csv',encoding='latin-1')
advertisementDF=advertisement[['ad_id','ad_title']]
advertisementDF.rename(columns={'ad_id':'ItemID','ad_title':'Title'},inplace=True)

C:\Users\Gandiv\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [18]:
def recommendProduct(predictions_df, userID, advertisementDF, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    #user_row_number = userID - 1 # UserID starts at 1, not 0
    user_row_number = predictions_df.index.get_loc(userID)
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    #sorted_user_predictions = predictions_df.iloc[predictions_df.index.get_loc(userID)].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(advertisementDF, how = 'left', left_on = 'ItemID', right_on = 'ItemID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    '''recommendations = (advertisementDF[~advertisementDF['ItemID'].isin(user_full['ItemID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'ItemID',
               right_on = 'ItemID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )'''
    samdf=advertisementDF[~advertisementDF['ItemID'].isin(user_full['ItemID'])].merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'ItemID',
               right_on = 'ItemID').dropna().rename(columns = {userID: 'Predictions'})
    grouped = samdf.groupby('Predictions').sum().reset_index()
    recommendations=grouped.sort_values('Predictions', ascending=False).iloc[:num_recommendations]

    return user_full, recommendations

already_rated, predictions = recommendProduct(predDF, 48, advertisementDF, sandf, 5)

User 48 has already rated 2 movies.
Recommending the highest 5 predicted ratings movies not already rated.


In [19]:
already_rated

,UserID,ItemID,Rating,Title
0,48,2401,5,Custom Shirt
1,48,2572,5,Fidget Spinner


In [20]:
predictions

,Predictions,ItemID
2691,0.005026,4784
2690,0.004830,4983
2689,0.004804,8951
2688,0.004774,8949
2687,0.004766,9592
